In [122]:
from itertools import chain
from collections import Counter
from nltk.tokenize import word_tokenize

# Define All Function

In [123]:
def get_data(filename):
    """Fungsi untuk melakukan load data pada file .tsv maupun .txt
    
    File yang digunakan yaitu file .tsv maupun file yang mampu dibuka
    menggunakan teks editor. Dalam file tersebut terdapat tag pembuka <kalimat>
    dan diakhiri tag penutup </kalimat> yang berfungsi sebagai penanda bahwa
    entitas tersebut termasuk ke dalam satu kalimat.
    
    Args:
        filename: string dari nama file yang akan diload datanya.
        
    Return:
        list dari kata-kata dan tags dengan index yang menunjukkan posisi kalimat tersebut.
    """
    
    # Load data dan buka sebagai file
    sentences = []
    tags = []
    with open(filename) as file:
        contents = file.readlines()

    # Hapus karakter \n yang tidak dibutuhkan
    contents = [content.strip() for content in contents]
    idx = 0
    while idx < len(contents):
        word = []
        tag = []
        # looping sampai menemukan pattern dengan awalan </kalimat
        while not contents[idx].startswith('</kalimat'):
            # kondisi jika menemukan sebuah data yang tidak memiliki awalan <kalimat
            if not contents[idx].startswith('<kalimat'):
                temp_word, temp_tag = contents[idx].split("\t")
                word.append(temp_word.lower())
                tag.append(temp_tag)
            idx += 1
        sentences.append(word)
        tags.append(tag)
        idx += 2
        
    return sentences, tags

In [124]:
def flatten(multi_list):
    """Mengembalikan list multi dimensi ke dalam list satu dimensi.
    
    Input list yang masuk di proses menggunakan method from_iterable dari
    package itertools dan mengembalikan object berupa generator yang iterable.
    
    Args:
        multi_list: list multi dimensi.
    
    Return:
        list satu dimensi yang nantinya digunakan untuk pemrosesan lebih lanjut.
    """
    
    return chain.from_iterable(multi_list)

In [125]:
def get_bigram(dataset):
    """Fungsi untuk membuat bigram dari list.
    
    Input list berisi string yang nantinya digenerate bigram dan mengembalikan
    objek berupa dictionary python.
    
    Args:
        dataset: list satu dimensi.
        
    Return:
        dictionary yang memuat bigram dan banyaknya bigram pada list tersebut.
    """
    
    bigrams = []
    for i in range(len(dataset)-1):
        bigrams.append((dataset[i], dataset[i+1]))
    count_bigrams = Counter(bigrams)
    
    return bigrams, dict(count_bigrams)

In [126]:
def get_prob_transition(dict_bigrams):
    """Fungsi untuk menghitung probabilitas dari dictionary perhitungan banyaknya muncul pasangan tag
    
    Tag diambil dan dihitung berdasarkan banyak kemunculan dibagi dengan total keseluruhan tag
    yang muncul.
    
    Args:
        dict_bigrams: dictionary berisi pasangan tag dan banyaknya kemunculan tag
    
    Return:
        tabel transisi yang berisi probabilitas kemunculan tag dan tag berikutnya
    
    """
    probs = []
    keys = []
    
    for states, value in count_bigrams.items():
        total_cnt = 0
        for sts in count_bigrams.keys():
            if states[0] == sts[0]:
                total_cnt += count_bigrams[sts]
        prob = value / total_cnt
        probs.append(prob)
        keys.append(states)
        
    probs_transition = dict(zip(keys,probs))
    return probs_transition

In [127]:
def remove_unneeded_states(count_bigrams):
    """Fungsi untuk menghilangkan states X -> <s>
    
    Dimana X adalah state apapun. Dihilangkan karena tidak dibutuhkan pada representasi transition
    table juga pada implementasi algoritma Viterbi
    
    Args:
        count_bigrams: dictionary berisi pasangan tag dan banyaknya kemunculan tag
    
    Return:
        dictionary berisi pasangan tags dan kemunculuannya tanpa beberapa pasang tags tertentu
    
    """
    terminated_states = []
    for states in count_bigrams.keys():
        if states[1] == "<s>":
            terminated_states.append(states)

    for states in terminated_states:
        del count_bigrams[states]
    return count_bigrams

In [128]:
def insert_sentence_beginning(tags_train):
    """Fungsi untuk menambahkan <s> pada setiap awal kalimat
    
    Args:
        tags_train: list dua dimensi berisikan list yang merepresentasikan satu kalimat
    
    Return:
        list dua dimensi yang terdapat penanda awal kalimat pada setiap kalimat
    
    """
    for sentence in tags_train:
        sentence.insert(0, '<s>')
    return tags_train

In [129]:
def get_emission_table(words, tags):
# Membuat representasi tabel emission probability dari HMM
    hidden_state = {}
    for word in words:
        # Jika word belum pernah ditemui maka akan digenerate
        # tagset berserta probabilitasnya
        if word not in hidden_state.keys():
            word_tags = []
            for idx, wrd in enumerate(words):
                if wrd == word :
                    word_tags.append(tags[idx])
            # Membuat dictionary berisi semua tagset dari sebauh word
            # pada korpus. Berserta jumlah kemunculan setiap tagsetnya
            tag_count = {}
            for tag in word_tags:
                if tag not in tag_count.keys():
                    tag_count[tag] = 1
                else:
                    tag_count[tag] += 1
            total = 0
            # Mengubah jumlah kemunculan menjadi probabilitas
            tag_prob = {}
            for count in tag_count.values():
                total += count
            for tagset in tag_count.keys():
                tag_prob[tagset] = tag_count[tagset]/total
            hidden_state[word] = tag_prob
        else:
            # Jika word sudah pernah ditemui maka akan dilewati
            continue
    return hidden_state

In [130]:
def load_dataset_emission():
    file_path = "../data/"
    file_name = "Indonesian_Manually_Tagged_Corpus_ID.tsv"
    words, tags = get_data(file_path + file_name)
    words = list(flatten(words))[:1000]
    tags = list(flatten(tags))[:1000]
    return words, tags

In [131]:
def get_tags_viterbi(sentence, emission_table, transition_table):
    viterbi_probs = []
    for i in range(1, len(sentence)):
        viterbi_probs.append("tmp")
    viterbi_path = []
    viterbi_prob_count = 0
    for idx, word in enumerate(sentence):
        if word == "<s>":
            continue
        else:
            for key_word, em_prob in emission_table.items():
                tmp_probs = []
                tmp_tags = []
                if key_word == word:
                    for tag, prob in em_prob.items():
                        tmp_tags.append(tag)
                        tmp_probs.append(prob)
                    if sentence[idx-1] == "<s>":
                        viterbi_path.append("<s>")
                        tmp = 0
                        tmp_idx_path = 0
                        tmp_idx_prob = 0
                        for sts, tr_prob in transition_table.items():
                            for idx, tag in enumerate(tmp_tags):
                                if viterbi_path[0] == sts[0] and tag == sts[1]:
                                    viterbi_prob = tmp_probs[idx] * tr_prob
                                    if tmp < viterbi_prob:
                                        tmp = viterbi_prob
                                        tmp_idx_path = idx
                                        viterbi_probs[viterbi_prob_count] = viterbi_prob
                                        viterbi_prob_count += 1
                        viterbi_path.append(tmp_tags[tmp_idx_path])
                    else:
                        tmp = 0
                        tmp_idx_path = 0
                        tmp_idx_prob = 0
                        for sts, tr_prob in transition_table.items():
                            for idx, tag in enumerate(tmp_tags):
                                if viterbi_path[-1] == sts[0] and tag == sts[1]:
                                    viterbi_prob = viterbi_probs[viterbi_prob_count-1] \
                                    * tmp_probs[idx] * tr_prob
                                    if tmp < viterbi_prob:
                                        tmp = viterbi_prob
                                        tmp_idx_path = idx
                                        viterbi_probs[viterbi_prob_count] = viterbi_prob
                                        viterbi_prob_count += 1
                                #else: backtrack       
                        viterbi_path.append(tmp_tags[tmp_idx_path])
                else:
                    break
    if len(viterbi_path) == 0:
        print("Kata pertama tidak terdapat di korpus")
        return viterbi_probs, viterbi_path
    return viterbi_probs[-1], viterbi_path

# Load and Separate Dataset

Load dan pisahkan dataset sebanyak 1000 data train dan 20 data test

In [132]:
filename = '../data/corpus.tsv'
sentences, tags = get_data(filename)

In [133]:
# mengambil 1020 data awal
data_train, data_test, tags_train, tags_test = sentences[:1000], sentences[1000:1020], tags[:1000], tags[1000:1020]

Tabel transisi dibuat berdasarkan data train yang telah diambil pada corpus

In [134]:
tags_train = insert_sentence_beginning(tags_train)
tags_train = list(flatten(tags_train))
bigrams, count_bigrams = get_bigram(tags_train)
count_bigrams = remove_unneeded_states(count_bigrams)

# Viterbi

In [135]:
words, tags = load_dataset_emission()
emission_table = get_emission_table(words, tags)
transition_table = get_prob_transition(count_bigrams)

In [142]:
data_test_final = []
for test in data_test:
    data_test_final.insert(0, "<s>")

In [144]:
for test_number, test in enumerate(data_test_final):
    print("Test Number " + str(test_number) + ": ")
    print(test)
    final_prob, final_path = get_tags_viterbi(test, emission_table, transition_table)
    print("Tag predictions: ")
    for tag in final_path:
        print(tag)

Test Number 0: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 1: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 2: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 3: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 4: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 5: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 6: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 7: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 8: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 9: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 10: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 11: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Number 12: 
<s>
Kata pertama tidak terdapat di korpus
Tag predictions: 
Test Numb